# Simple Double Pendulum
NOTE: $\Delta\theta = \theta_1-\theta_2$. Solving equations
$$
\newcommand{\Dtheta}{\Delta\theta}

\begin{align*}
    \dot\theta_1 &= \omega_1 \\
    \dot\theta_2 &= \omega_2 \\
    \dot\omega_1 &= \frac{-m_2(L_1\dot\theta_1^2\sin\Dtheta\cos\Dtheta + L_2\dot\theta_2^2\sin\Dtheta)}{m_1L_1+m_2L_2\sin\Dtheta} +
     \frac{ g( -(m_1+m_2)\sin\theta_1 + m_2\cos\Dtheta\sin\theta_2 )}{m_1L_1+m_2L_2\sin\Dtheta}  \\
\\
    \dot\omega_2 &= \frac{ (m_1+m_2)L_1\dot\theta_1^2\sin\Dtheta + m_2L_2\dot\theta_2^2\sin\Dtheta\cos\Dtheta}{m_1L_2 + m_2L_2\sin\Dtheta} + \frac{(m_1+m_2)g(\sin\theta_1\cos\Dtheta - \sin\theta_2) }{m_1L_2 + m_2L_2\sin\Dtheta}
\end{align*}
$$
External libraries used:
- `numpy`        - for vector operations
- `matplotlib`   - for visualization
- `tqdm`         - for progress bar

In [ ]:
import numpy as np
from numpy._typing import NDArray
import matplotlib.pyplot as plt
from numpy import sin, cos
from functools import partial
from typing import Callable

A class to represent Double Pendulum State.

Contains a state vector $\mathbf Y$ where $\mathbf Y = [\theta_1,\, \theta_2,\, \omega_1,\, \omega_2]$.

In [ ]:
class DoublePendulum:
    def __init__(self, state_vector, masses, lengths):
        self.state_vector = state_vector
        self.m1 = masses[0]
        self.m2 = masses[1]
        self.l1 = lengths[0]
        self.l2 = lengths[1]

Implementing the $\phi$ function. To solve equation of the form
$$ \frac{\mathrm d\mathbf Y}{\mathrm dt} = \phi(\mathbf Y)$$
Using RK4 method from before

In [ ]:
def phi(dp_state: DoublePendulum, Y: NDArray) -> NDArray:
    m1, m2 = dp_state.m1, dp_state.m2
    L1, L2 = dp_state.l1, dp_state.l2
    theta1, theta2, omega1, omega2 = Y
    dtheta = theta1 - theta2
    Y_n = np.empty(4)

    Y_n[0] = omega1
    Y_n[1] = omega2

    # omega1
    Y_n[2] = ( -m2*(L1*omega1**2*sin(dtheta)*cos(dtheta) + L2*omega2**2*sin(dtheta)) + g*(-(m1+m2)*sin(theta1) + m2*cos(dtheta)*sin(theta2)) ) / (m1*L1 + m2*L1*sin(dtheta)**2)

    # omega2
    Y_n[3] = ( (m1+m2)*L1*omega1**2*sin(dtheta) + m2*L2*omega2**2*sin(dtheta)*cos(dtheta) + (m1+m2)*g*(sin(theta1)*cos(dtheta) - sin(theta2)) ) / (m1*L2 + m2*L2*sin(dtheta)**2)

    return Y_n

def rk4_step(y_i: float, dt: float, f: Callable):
    k1 = f(y_i)
    k2 = f(y_i + dt*k1/2)
    k3 = f(y_i + dt*k2/2)
    k4 = f(y_i + dt*k3)
    y_n = y_i + dt/6 * (k1 + 2*k2 + 2*k3 + k4)
    return y_n 

def solve_pendulum(dp_state: DoublePendulum, dt, N_steps) -> NDArray:
    Y_i = np.empty((N_steps+1, 4))
    Y_i[0] = dp_state.state_vector
    partial_phi = partial(phi, dp_state)
    for i in range(N_steps):
        Y_i[i+1] = rk4_step(Y_i[i], dt, partial_phi)
    return Y_i


Initial conditions.
For the parametric plot given in PPT, use $\theta_1 = 0.1$ and $\theta_2 = 0.1$, and $t_n=50$.

In [ ]:
theta1_0, theta2_0 = 90, 90
omega1_0, omega2_0 = 0, 0
g = 9.81
initial_state_vector = np.radians(np.array([theta1_0, theta2_0, omega1_0, omega2_0]))
dp_state = DoublePendulum(
    state_vector=initial_state_vector,
    masses=[10, 20],
    lengths=[10, 20]
)

In [ ]:
dt = 0.01
t_n = 250    # in seconds
N = int(t_n/dt)

Y = solve_pendulum(dp_state, dt, N)

Plotting,

In [ ]:
theta1, theta2, omega1, omega2 = Y.T
x1 = dp_state.l1*sin(theta1)
y1 = -dp_state.l1*cos(theta1)
x2 = dp_state.l1*sin(theta1) + dp_state.l2*sin(theta2)
y2 = -dp_state.l1*cos(theta1) - dp_state.l2*cos(theta2)
plt.rcParams.update({'font.size': 14})

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title("Angles plotted as parametric plot")
ax.plot(theta1, theta2)
ax.set_xlabel(r"$\theta_1$")
ax.set_ylabel(r"$\theta_2$")
fig.savefig("5-dp-phase-t1-t2", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
# ax.plot(x1, y1)
ax.plot(x2, y2, color="#ffd48f")

ax.plot([-100, 100], [0, 0], '-', color="#cccccc")
ax.plot([0, 0], [-100, 100], '-', color="#cccccc")
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Evolution of pendulum over time")
string, = ax.plot([0, x1[-1], x2[-1]], [0, y1[-1], y2[-1]], '-o', color="blue")
fig.savefig("6-dp-evolution")

### Lyapunov Exponent Analysis
From the algorithm used in Lorenz Equations, we can visualize the Lyapunov Exponents for multiple initial angles

In [ ]:
import copy

In [ ]:
def compute_largest_lyapunov(dp_state: DoublePendulum) -> float:
    d0 = 1e-6
    t_total = 10  # in seconds
    t_step = 1
    N_steps = int(t_total/t_step)
    N_iter = int(t_step/dt) # For Pendulum integration

    state1 = copy.deepcopy(dp_state) 
    state2 = copy.deepcopy(dp_state)
    step_vector = np.array([d0, 0, 0, 0])
    lyapunov_list = []

    for _ in range(N_steps):
        state2.state_vector = state1.state_vector + step_vector

        Y1 = solve_pendulum(state1, dt, N_iter)
        Y2 = solve_pendulum(state2, dt, N_iter)

        Y_new = Y1[-1]
        Y_pert = Y2[-1]

        distance_vector = Y_new - Y_pert
        distance = np.linalg.norm(distance_vector)
        lyapunov = np.log(distance / d0 + 1e-12)
        lyapunov_list.append(lyapunov)

        step_vector = distance_vector/distance*d0
        state1.state_vector = Y_new

    return sum(lyapunov_list)/(N_steps * t_step)

Test value

In [ ]:
compute_largest_lyapunov(dp_state)

Code for plotting the Lyapunov grid. 

In [ ]:
from tqdm.notebook import tqdm  # For progress bar

Higher $N_{\theta}$ -> Higher resolution.

Higher $t_{total}$ (in `compute_largest_lyapunov`) -> Higher accuracy of $\lambda$

Est. Time: 30 mins

Run the code again : Use linspace

In [ ]:
N_theta = 100
test1 = np.arange(0, 2*np.pi, (2*np.pi/N_theta))
test2 = np.linspace(0, 2*np.pi, N_theta)

In [ ]:
N_theta = 100
theta_diff = 2*np.pi/N_theta
initial_theta1_array = np.arange(0, 2*np.pi, theta_diff)
initial_theta2_array = np.arange(0, 2*np.pi, theta_diff)

lyapunov_matrix = []

for i in tqdm(range(N_theta)):
    lyapunov_array = np.empty(N_theta)
    for j in tqdm(range(N_theta), leave=False):
        t1 = initial_theta1_array[j]
        t2 = initial_theta2_array[i]
        dp_state_initial = DoublePendulum(
            state_vector=np.array([t1, t2, 0, 0]),
            masses = [dp_state.m1, dp_state.m2],
            lengths = [dp_state.l1, dp_state.l2]
        )
        lyapunov_exp = compute_largest_lyapunov(dp_state_initial)
        lyapunov_array[j] = lyapunov_exp
    lyapunov_matrix.append(lyapunov_array)

In [ ]:
lyapunov_matrix = np.array(lyapunov_matrix)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
cmap = ax.pcolormesh(initial_theta2_array, initial_theta1_array, lyapunov_matrix.T, cmap="plasma", vmin=0, vmax=1)
ax.set_title(r"Lyapunov Exponents for different $\theta_1$ and $\theta_2$")
ax.set_xlabel(r"$\theta_2$")
ax.set_ylabel(r"$\theta_1$")
fig.colorbar(cmap, label=r"$\lambda$")
fig.savefig("7-lyapunov-exp-graph")